In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import matplotlib.pylab as pylab
import cartopy.feature as cfeature
from scipy.interpolate import griddata
import scipy.stats as stats
import xarray as xr
import pdb
import glob
import itertools
from utils import lappi_constants as cnst

pylab.rcParams['figure.figsize'] = (50., 50.)
%matplotlib inline

In [ ]:
xr.open_dataarray(cnst.)